In [32]:
from gen_data_and_simulate_drift import DriftIntensity, DriftSimulator, GenerateFakeData
from training_script import Classifier

In [33]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import OrdinalEncoder

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
DATA_PATH = '../datasets/'
MODEL_PATH = '../models/'

### Exploring the generator class

In [164]:
pokemon_test_data='bank_num.csv'
df = pd.read_csv(DATA_PATH + pokemon_test_data )

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        11162 non-null  int64
 1   job        11162 non-null  int64
 2   marital    11162 non-null  int64
 3   education  11162 non-null  int64
 4   default    11162 non-null  int64
 5   balance    11162 non-null  int64
 6   housing    11162 non-null  int64
 7   loan       11162 non-null  int64
 8   contact    11162 non-null  int64
 9   day        11162 non-null  int64
 10  month      11162 non-null  int64
 11  duration   11162 non-null  int64
 12  campaign   11162 non-null  int64
 13  pdays      11162 non-null  int64
 14  previous   11162 non-null  int64
 15  poutcome   11162 non-null  int64
 16  deposit    11162 non-null  int64
dtypes: int64(17)
memory usage: 1.4 MB


In [301]:
SAMPLE_SIZE=1000
target = 'default'
genertor_fake_data = GenerateFakeData(path_ref_data=DATA_PATH+'bank_num.csv', sample_size=SAMPLE_SIZE, target=target, model_name = 'bank_generator.pkl')

default


In [302]:
genertor_fake_data.df_ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  int64 
 2   marital    11162 non-null  int64 
 3   education  11162 non-null  int64 
 4   balance    11162 non-null  int64 
 5   housing    11162 non-null  int64 
 6   loan       11162 non-null  int64 
 7   contact    11162 non-null  int64 
 8   day        11162 non-null  int64 
 9   month      11162 non-null  int64 
 10  duration   11162 non-null  int64 
 11  campaign   11162 non-null  int64 
 12  pdays      11162 non-null  int64 
 13  previous   11162 non-null  int64 
 14  poutcome   11162 non-null  int64 
 15  deposit    11162 non-null  int64 
 16  default    11162 non-null  object
dtypes: int64(16), object(1)
memory usage: 1.4+ MB


In [303]:
genertor_fake_data.df_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 499
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        1000 non-null   int64 
 1   job        1000 non-null   int64 
 2   marital    1000 non-null   int64 
 3   education  1000 non-null   int64 
 4   balance    1000 non-null   int64 
 5   housing    1000 non-null   int64 
 6   loan       1000 non-null   int64 
 7   contact    1000 non-null   int64 
 8   day        1000 non-null   int64 
 9   month      1000 non-null   int64 
 10  duration   1000 non-null   int64 
 11  campaign   1000 non-null   int64 
 12  pdays      1000 non-null   int64 
 13  previous   1000 non-null   int64 
 14  poutcome   1000 non-null   int64 
 15  deposit    1000 non-null   int64 
 16  default    1000 non-null   object
dtypes: int64(16), object(1)
memory usage: 140.6+ KB


In [304]:
genertor_fake_data.df_samples.head()

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,default
0,44,1,1,1,348,0,1,2,30,8,889,1,365,3,0,1,no
1,42,9,1,1,530,0,0,1,11,6,86,0,4,0,2,0,no
2,50,9,0,0,1481,1,0,0,12,9,113,1,0,0,3,0,no
3,31,6,1,1,757,0,0,0,18,8,446,1,7,0,2,0,no
4,29,0,1,1,-272,0,0,0,29,0,321,4,163,1,2,0,no


In [306]:
sampled_data = genertor_fake_data.get_dataclass_sampling()

In [307]:
sampled_data.train_data.describe()

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.00000,800.000000,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,40.296250,3.98250,0.732500,0.806250,639.498750,0.23250,0.125000,0.438750,16.16625,5.051250,337.666250,2.395000,30.208750,0.377500,2.238750,0.216250
std,11.222563,3.31148,0.697533,0.790248,1987.421832,0.42269,0.330926,0.737854,8.83834,2.954579,294.154821,3.115864,94.286997,1.578166,0.835237,0.411944
min,19.000000,-1.00000,0.000000,0.000000,-7168.000000,0.00000,0.000000,0.000000,-3.00000,-1.000000,-64.000000,-3.000000,-84.000000,-1.000000,0.000000,0.000000
25%,32.000000,1.00000,0.000000,0.000000,-229.000000,0.00000,0.000000,0.000000,9.00000,3.000000,129.750000,1.000000,-14.000000,0.000000,2.000000,0.000000
50%,39.000000,4.00000,1.000000,1.000000,168.000000,0.00000,0.000000,0.000000,16.00000,5.000000,257.000000,2.000000,3.000000,0.000000,2.000000,0.000000
75%,47.000000,7.00000,1.000000,1.000000,831.750000,0.00000,0.000000,1.000000,23.00000,7.000000,456.500000,3.000000,25.000000,0.000000,3.000000,0.000000
max,86.000000,12.00000,2.000000,3.000000,18869.000000,1.00000,1.000000,2.000000,38.00000,12.000000,1897.000000,25.000000,782.000000,22.000000,3.000000,1.000000


In [308]:
sampled_data.test_data.describe()

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,40.805000,4.000000,0.730000,0.855000,787.530000,0.195000,0.085000,0.385000,15.605000,4.950000,341.440000,2.440000,42.090000,0.495000,2.130000,0.215000
std,12.089573,3.220537,0.713897,0.864704,2051.906656,0.397195,0.279582,0.720675,9.305479,3.166645,287.167849,3.416837,101.072151,1.517064,0.892796,0.411853
min,21.000000,-1.000000,0.000000,0.000000,-6453.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-54.000000,-1.000000,-75.000000,-1.000000,0.000000,0.000000
25%,31.750000,1.000000,0.000000,0.000000,-197.500000,0.000000,0.000000,0.000000,7.000000,2.000000,115.250000,1.000000,-7.000000,0.000000,2.000000,0.000000
50%,39.000000,3.000000,1.000000,1.000000,301.500000,0.000000,0.000000,0.000000,15.500000,5.000000,259.500000,1.000000,8.000000,0.000000,2.000000,0.000000
75%,48.000000,7.000000,1.000000,1.000000,1013.000000,0.000000,0.000000,0.250000,23.000000,8.000000,505.500000,3.000000,35.500000,0.000000,3.000000,0.000000
max,87.000000,11.000000,2.000000,3.000000,13227.000000,1.000000,1.000000,3.000000,35.000000,11.000000,1238.000000,25.000000,502.000000,11.000000,3.000000,1.000000


In [295]:
sampled_data.list_num_col

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'deposit']

In [321]:
pok_classifier = Classifier(df_train=sampled_data.train_data,
                 num_features=sampled_data.list_num_col,
                 cat_features=None,
                 target=target,
                pkl_file_path=MODEL_PATH + f'class_{target}_model.pkl',
                iterations = 200, depth = None, learning_rate = 0.03)

In [322]:
pok_classifier.train()

0:	learn: 0.6814581	total: 1.28ms	remaining: 255ms
1:	learn: 0.6726310	total: 2.13ms	remaining: 211ms
2:	learn: 0.6629441	total: 2.75ms	remaining: 180ms
3:	learn: 0.6545960	total: 3.4ms	remaining: 166ms
4:	learn: 0.6476254	total: 4.76ms	remaining: 186ms
5:	learn: 0.6400082	total: 5.42ms	remaining: 175ms
6:	learn: 0.6336210	total: 6.08ms	remaining: 168ms
7:	learn: 0.6259030	total: 6.76ms	remaining: 162ms
8:	learn: 0.6191754	total: 7.43ms	remaining: 158ms
9:	learn: 0.6137379	total: 8.03ms	remaining: 153ms
10:	learn: 0.6072295	total: 8.66ms	remaining: 149ms
11:	learn: 0.6002511	total: 9.27ms	remaining: 145ms
12:	learn: 0.5957597	total: 9.82ms	remaining: 141ms
13:	learn: 0.5888169	total: 10.4ms	remaining: 138ms
14:	learn: 0.5846006	total: 11ms	remaining: 135ms
15:	learn: 0.5785326	total: 11.5ms	remaining: 133ms
16:	learn: 0.5728251	total: 12.1ms	remaining: 130ms
17:	learn: 0.5679640	total: 12.7ms	remaining: 129ms
18:	learn: 0.5635807	total: 13.3ms	remaining: 126ms
19:	learn: 0.5587290	tota

In [317]:
pok_classifier.serialize()

Error occurred while pickling: Can't pickle <class 'training_script.Classifier'>: it's not the same object as training_script.Classifier


In [323]:
prediction = pok_classifier.predict(sampled_data.train_data.drop(target, axis = 1))
#prediction = [1 if e=='True' else 0 for e in prediction]
prediction[:4]

array(['no', 'yes', 'no', 'yes'], dtype=object)

In [324]:
(prediction==sampled_data.train_data[target].values).mean()

0.90125

In [325]:
prediction = pok_classifier.predict(sampled_data.test_data.drop(target, axis = 1))
#prediction = [1 if e=='True' else 0 for e in prediction]
(prediction==sampled_data.test_data[target].values).mean()

0.765

### Analyzing the drift

In [349]:
from gen_data_and_simulate_drift import get_shift_coef

In [363]:
drift_sim_info = DriftSimulator(sampled_data, nb_cols_to_drift=1)

number of columns to drift is : 1
select random column to drift ...
Drifting column job


In [365]:
df_test_drifted = drift_sim_info.get_test_data_drifted()

In [367]:
df_test_drifted.job.describe()

count    200.000000
mean       6.265000
std        4.735706
min       -4.000000
25%        3.000000
50%        6.000000
75%        9.250000
max       22.000000
Name: job, dtype: float64

In [369]:
prediction = pok_classifier.predict(df_test_drifted.drop(target, axis = 1))
#prediction = [1 if e=='True' else 0 for e in prediction]
(prediction==sampled_data.test_data[target].values).mean()

0.74

In [368]:
sampled_data.test_data.job.describe()

count    200.000000
mean       4.000000
std        3.220537
min       -1.000000
25%        1.000000
50%        3.000000
75%        7.000000
max       11.000000
Name: job, dtype: float64

### Creating marketing dataset

In [66]:
import os

In [69]:
os.path.exists(DATA_PATH + 'bank.csv')

True

In [177]:
bank = pd.read_csv(DATA_PATH + 'bank.csv')

In [178]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [179]:
cat_cols = bank.select_dtypes('O').columns
cat_cols = cat_cols.drop('default')
cat_cols

Index(['job', 'marital', 'education', 'housing', 'loan', 'contact', 'month',
       'poutcome', 'deposit'],
      dtype='object')

In [180]:
oe = OrdinalEncoder(dtype=np.int64, handle_unknown='use_encoded_value', unknown_value=-1)

In [181]:
bank[cat_cols].head()

,job,marital,education,housing,loan,contact,month,poutcome,deposit
0,admin.,married,secondary,yes,no,unknown,may,unknown,yes
1,admin.,married,secondary,no,no,unknown,may,unknown,yes
2,technician,married,secondary,yes,no,unknown,may,unknown,yes
3,services,married,secondary,yes,no,unknown,may,unknown,yes
4,admin.,married,tertiary,no,no,unknown,may,unknown,yes


In [182]:
bank[cat_cols] = oe.fit_transform(bank[cat_cols])

In [183]:
bank[cat_cols].head()

,job,marital,education,housing,loan,contact,month,poutcome,deposit
0,0,1,1,1,0,2,8,3,1
1,0,1,1,0,0,2,8,3,1
2,9,1,1,1,0,2,8,3,1
3,7,1,1,1,0,2,8,3,1
4,0,1,2,0,0,2,8,3,1


In [184]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,0,1,1,no,2343,1,0,2,5,8,1042,1,-1,0,3,1
1,56,0,1,1,no,45,0,0,2,5,8,1467,1,-1,0,3,1
2,41,9,1,1,no,1270,1,0,2,5,8,1389,1,-1,0,3,1
3,55,7,1,1,no,2476,1,0,2,5,8,579,1,-1,0,3,1
4,54,0,1,2,no,184,0,0,2,5,8,673,2,-1,0,3,1


In [185]:
joblib.dump(oe, MODEL_PATH+ 'bank_ordinal_encoder.pkl')

['../models/bank_ordinal_encoder.pkl']

In [186]:
bank.to_csv(DATA_PATH + 'bank_num.csv', index = False)

In [187]:
bank.dtypes

age           int64
job           int64
marital       int64
education     int64
default      object
balance       int64
housing       int64
loan          int64
contact       int64
day           int64
month         int64
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome      int64
deposit       int64
dtype: object